# Understanding the Kaggle Data
The dataset acquired from [Kaggle](https://www.kaggle.com/code/martinkk5575/language-detection/data) contains words from several different languages. The noise contained in the dataset are duplicate words. To reduce this noise, the words will be broken down into single and double characters, then rated based on how often they show up in that respective language.

In [102]:
#imports
import pandas as pd
import numpy as np

fileName = "dataset.csv"
data = pd.read_csv(fileName)

data

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
...,...,...
21995,hors du terrain les années et sont des année...,French
21996,ใน พศ หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...,Thai
21997,con motivo de la celebración del septuagésimoq...,Spanish
21998,年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...,Chinese


In [46]:
import numpy as np
from sklearn.model_selection import train_test_split

X=data['Text']
y=data['language']

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Save the languages into a DataFrame that we aren't modifying
languages = set(y)

# Kaggle Resources
Kaggle Found that there is a performance rate of 97% of Correctly matching characters using a mixture of the top 50 single and double characters used in their words dataset. This is code extracted from their findings


In [104]:
#This Command from Kaggle connects the character features to their specific language
# Aggregate Unigrams per language
def train_lang_dict(X_raw_counts, y_train):
    lang_dict = {}
    for i in range(len(y_train)):
        lang = y_train[i]
        v = np.array(X_raw_counts[i])
        if not lang in lang_dict:
            lang_dict[lang] = v
        else:
            lang_dict[lang] += v
            
    # to relative
    for lang in lang_dict:
        v = lang_dict[lang]
        lang_dict[lang] = v / np.sum(v)
        
    return lang_dict

# Collect relevant chars per language to Display them
def getRelevantCharsPerLanguage(features, language_dict, significance=1e-5):
    relevantCharsPerLanguage = {}
    for lang in languages:
        chars = []
        relevantCharsPerLanguage[lang] = chars
        v = language_dict[lang]
        for i in range(len(v)):
            if v[i] > significance:
                chars.append(features[i])
    return relevantCharsPerLanguage

In [106]:
#This will create a list of sigle and double characters from the top 1% of to be used as features
from sklearn.feature_extraction.text import CountVectorizer

top1PrecentMixtureVectorizer = CountVectorizer(analyzer='char', ngram_range=(1,2), min_df=1e-2)
X_top1Percent_train_raw = top1PrecentMixtureVectorizer.fit_transform(X_train)
X_top1Percent_test_raw = top1PrecentMixtureVectorizer.transform(X_test)

language_dict_top1Percent = train_lang_dict(X_top1Percent_train_raw.toarray(), y_train.values)

top1PercentFeatures = top1PrecentMixtureVectorizer.get_feature_names_out()

In [107]:
def getRelevantGramsPerLanguage(features, language_dict, top=50):
    relevantGramsPerLanguage = {}
    for lang in languages:
        chars = []
        relevantGramsPerLanguage[lang] = chars
        v = language_dict[lang]
        sortIndex = (-v).argsort()[:top]
        for i in range(len(sortIndex)):
            chars.append(features[sortIndex[i]])
    return relevantGramsPerLanguage

top50PerLanguage_dict = getRelevantGramsPerLanguage(top1PercentFeatures, language_dict_top1Percent)

# top50
allTop50 = []
for lang in top50PerLanguage_dict:
    allTop50 += set(top50PerLanguage_dict[lang])

top50 = list(set(allTop50))
    
print('All items:', len(allTop50))
print('Unique items:', len(top50))

All items: 1100
Unique items: 564


With this, we have 564 unique features which we can start training a model on.

In [115]:
pd.DataFrame(top50).transpose()

,0,1,2,3,4,5,6,7,8,9,...,554,555,556,557,558,559,560,561,562,563
0,ี่,・,ق,न,的,ро,他,g,t,پ,...,る,த்,ہ,о,ț,är,as,่อ,а,е


In [8]:
#Create these arrays into dictonaries
english_alpha = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
estonian_alpha = [A, B, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, Š, Z, Ž, T, U, V, Õ, Ä, Ö, Ü]
swedish_alpha = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, å, ä, ö]
thai_alpha = [ก, ข, ค, ฅ, ฆ, ง, จ, ฉ, ช, ฌ, ญ, ฎ, ฏ, ฐ, ฑ, ฒ, ณ, ด, ต, ถ, ท, ธ, น, บ, บ, ผ, ฝ, พ, ฟ, ภ, 
               ม, ย, ร, ล, ว, ศ, ษ, ส, ห, ฬ, อ, ฮ] 
tamil_alpha = [அ, ஆ, இ, ஈ, உ, ஊ, எ, ஏ, ஐ, ஒ, ஓ, ஔ, க, ங, ச, ஞ, ட, ண, த, ந, ன, ப, ம, ய, ர, ற, ல, ள, ழ, வ]
dutch_alpha = english_alpha
japanese_alpha = [ぁ, あ, ぃ, い, ぅ, う, ぇ, え, ぉ, お, か, が, き, ぎ, く, ぐ, け, げ, こ, ご, さ, ざ, し, じ, す, ず,
                  せ, ぜ, そ, ぞ, た, だ, ち, ぢ, っ, つ, づ, て, で, と, ど, な, に, ぬ, ね, の, は, ば, ぱ, ひ, び, ぴ,
                  ふ, ぶ, ぷ, へ, べ, れ, る, り, ら, よ, ょ, ゆ, ゅ, や, ゃ, も, め, む, み, ま, ぽ, ぼ, ほ, ぺ, ろ, ゎ,
                  わ, ゐ, ゑ, を, ん, ゔ, ゕ, ゖ,  ゚, ゛, ゜, ゝ, ゞ, ゟ, ゠, ァ, ア, サ, ゴ, コ, ゲ, ケ, グ, ク, ギ, キ,
                  ガ, カ, オ, ォ, エ, ェ, ウ, ゥ, イ, ィ, ザ, シ, ジ, ス, ズ, セ, ゼ, ソ, ゾ, タ ,ダ ,チ ,ヂ, ッ, ツ, ヅ,
                  テ, デ, ト, ホ, ペ, ベ, ヘ, プ, ブ, フ, ピ, ビ, ヒ, パ, バ, ハ, ノ, ネ, ヌ, ニ, ナ, ド, ボ, ポ, マ, ミ, 
                  ム, メ, モ, ャ, ヤ, ュ, ユ, ョ, ヨ, ラ, リ, ル, レ, ロ, ヮ, ㍿, ㍐, ヿ, ヾ, ヽ, ー, ・, ヺ, ヹ, ヸ, ヷ,
                  ヶ, ヵ, ヴ, ン, ヲ, ヱ, ヰ, ワ]
turkish_alpha = [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, r, s, t, u, v, y, z, ç, ğ, ı, İ, î, ö, ş, ü]
latin_alpha = english_alpha
urdu_alpha = [ش,س,ژ,ز,ڑ,ر,ذ,ڈ,د,خ,ح,چ,
              ج,ث,ٹ,ت,پ,ب,آ,ا,ے,ی,ھ,ہ,و,ں,ن,م,ل,گ,ک,ق,ف,غ,ع,ظ,ط,ض,ص]
indonesian_alpha = english_alpha
portuguese_alpha = [ç, á, é, í, ó, ú, â, ê, ô, ã, õ, à, è, ì, ò, ù, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
french_alpha = [ç, é, â, ê, î, ô, û, à, è, ì, ò, ù, ë, ï, ü, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
chinese_alpha = [胡, 赛, 尼, 本, 人, 和, 小, 说, 的, 主, 人, 公, 阿, 米, 尔, 一, 样, 都, 是, 出, 生, 在, 阿, 富, 汗, 首, 都, 
                 喀, 布, 尔, 少, 年, 时, 代, 便, 离, 开, 了, 这, 个, 国, 家, 。, 胡, 赛, 尼, 直, 到, 年, 小, 说, 出, 版, 之, 
                 后, 才, 首, 次, 回, 到, 已, 经, 离, 开, 年, 的, 祖, 国, 。, 他, 在, 苏, 联, 入, 侵, 时, 离, 开, 了, 阿, 富, 
                 汗, 而, 他, 的, 很, 多, 童, 年, 好, 友, 在, 阿, 富, 汗, 生, 活, 在, 他, 们, 出, 发, 之, 前, 罗, 伯, 特, 伊,
                 达, 尔, 文, 卷, 查, 尔, 斯, 赖, 尔, 所, 著, 地, 质, 学, 原, 理, 在, 南, 美, 他, 得, 到, 第, 卷, 该, 书, 将, 
                 地, 形, 地, 貌, 解, 释, 为, 漫, 长, 历, 史, 时, 间, 渐, 进, 演, 变, 的, 的, 结, 果, 当, 他, 旅, 程, 的, 第, 
                 站, 抵, 达, 圣, 地, 亚, 哥, 佛, 得, 角, 的, 时, 候, 达, 尔, 文]
korean_alpha = [ᄁ,ᄂ,ᄃ,ᄄ,ᄅᄆᄇ,ᄈ,ᄉ,ᄊ,ᄋ,ᄌᄍ,ᄎ,ᄏ,ᄐ,ᄑᄒ,아,악,안,알,암,압,앙,앞애,액,앵야,얀,약,양,얘,어,억,
                언,얼,엄,업,엉,에,여,역,연,열,염,엽,영,예,ᄀ,여,역,연,열,염,엽,영,예,오,옥,온,올,옴,옹,와,완,왈,왕,왜,외,왼,
                요,욕,용,우,욱,운,울,움,웅,워,원,월,위,유,육,윤,율,융,윷,으,은,을,음읍,응,의,이,익,인,일,임,입,잉,잎]
hindi_alpha = [ऄ, अ, आ, इ, ई, उ, ऊ, ऋ, ऌ, ऍ, ऎ, ए, ऐ, ऑ, ऒ, ओ, औ, क, ख, ग, घ, ङ, च, छ, ज, झ, प, ऩ, न, ध, द, 
               थ, त, ण, ढ, ड, ठ, ट, ञ, फ, ब, भ, म, य, र, ऱ, ल, ळ, ऴ, व, श, ष, ४, ३, २, १, ०, ॥, ।, ॡ, ॠ, ॐ, ऽ, 
               ह, स, ५, ६, ७, ॲ, ॳ, ॴ, ॵ, ॶ, ॷ, ॹ, ॺ, ॻ, ॼ, ॾ, ॿ, ೱऀँं, ः, ऺ, ऻ, ा, ि, ी, ॎ, ॏॕैेॣॢ, ॗ]
spanish_alpha = [á, é, í, ó, ú, ñ, ü, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
pushto_alpha = [,ﺏ ,پ ,ﺕ ,ټ ,ﺙ ,ﺝ ,چ ,ﺡ ,ﺥ ,څ ,ځ ,ﺩ ,ډ ,ﺫ ,ﺭ ,ړ ,ﺯ ,ژ ,ږ ,ﺱ ,ﺵ ,ښ ,ﺹ ,ﺽ ,ﻁ ,ﻅ ,ﻉ ,ﻍ ,ﻑ ,ﻕ ,ک ,ګ ,ﻝ ,ﻡ ,ﻥ ,ڼ, ,ﻭ ,ه ,ۀ ,ي ,ې ,ی ,ۍ ,ئ]
persian_alpha = [,ش,س,ژ,ز,ر,ذ,د,خ,ح,چ,ج,ث,ت,پ,ب,آ,ا,ص,ض,ط,ظ,ع,غ,ف,ق,ک,گ,ل,م,ن,و,ه,ی]
romanian_alpha = [ă, â, î, ș, ş, ț, ţ, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]
russian_alpha = [б, в, г, д, ж, з, к, л, м, н, п, р, с, т, ф, х, ц, ч, ш, щ, а, е, ё, и, о, у, ы, э, ю, я, й]
arabic_alpha = [ش,س,ز,ر,ذ,د,خ,ح,ج,ث,ت,ب,ا,ء,ي,و,ه,ن,م,ل,ك,ق,ف,غ,ع,ظ,ط,ض,ص]


SyntaxError: invalid character '゚' (U+309A) (3693930689.py, line 11)

In [20]:
#functions:
def main(string):
    print("This program will calculate average word length!")
    #s = raw_input ("Enter a phrase: ")
    # use temporary test string instead of input
    words = string.split()
    wordCount = len(words)
    # start with zero characters
    ch = 0
    for word in words:
        # add up characters
        ch += len(word)  
    avg = ch / wordCount
    return avg


In [21]:
string_1 = "Hello my name is Jesse"
main(string_1)

This program will calculate average word length!


3.6

In [22]:
#function to calculate the average sentence length across a piece of text.
def avg_sentence_len(text):
  sentences = text.split(".") #split the text into a list of sentences.
  words = text.split(" ") #split the input text into a list of separate words
  if(sentences[len(sentences)-1]==""): #if the last value in sentences is an empty string
    average_sentence_length = len(words) / len(sentences)-1
  else:
    average_sentence_length = len(words) / len(sentences)
  return average_sentence_length #returning avg length of sentence
  
ans = avg_sentence_len("I am going.to see you later") #function call
print(ans) #printing result

3.0
